In [1]:
import pyspark
import os

# Use graphframes and create new spark context
SUBMIT_ARGS = "--packages graphframes:graphframes:0.7.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

conf = pyspark.SparkConf()
sc = pyspark.SparkContext.getOrCreate(conf=conf)
print(sc._conf.getAll())
import sys
pyfiles = str(sc.getConf().get(u'spark.submit.pyFiles')).split(',')
sys.path.extend(pyfiles)
from graphframes import *

[('spark.driver.host', 'Vincent-PC'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1588897833965'), ('spark.app.name', 'pyspark-shell'), ('spark.jars', 'file:///C:/Users/vince/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///C:/Users/vince/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'), ('spark.submit.pyFiles', 'C:/Users/vince/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,C:/Users/vince/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'), ('spark.driver.port', '60783'), ('spark.rdd.compress', 'True'), ('spark.files', 'file:///C:/Users/vince/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///C:/Users/vince/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.repl.local.jars', 'file:///C:/Users/vince/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///C:/Users/vi

In [2]:
# create Sqlcontext to load in data into Spark dataframes
sql = pyspark.SQLContext(sc)
trip_df  = sql.read.option("header", "true").csv('trip.csv')
station_df = sql.read.option("header", "true").csv('station.csv')

In [3]:
trip_df.show()

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|      70|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...|              27|8/29/2013 10:17|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|8/29/2013 11:29|  San Jose City Hall|      

In [4]:
# convert 10 rows to pandas df for pretty print
trip_df.limit(10).toPandas()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103
5,4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109
6,4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112
7,4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,95112
8,4760,113,8/29/2013 17:01,South Van Ness at Market,66,8/29/2013 17:03,South Van Ness at Market,66,553,Subscriber,94103
9,4258,114,8/29/2013 11:33,San Jose City Hall,10,8/29/2013 11:35,MLK Library,11,107,Subscriber,95060


In [5]:
station_df.show()

+---+--------------------+------------------+-------------------+----------+------------+-----------------+
| id|                name|               lat|               long|dock_count|        city|installation_date|
+---+--------------------+------------------+-------------------+----------+------------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|    San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|    San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|    San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|    San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|    San Jose|         8/7/2013|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|        15|    San Jose|         8/7/2013|
|  8| San Salvador at 1st|  

In [6]:
# Create vertices and edges of graph
stationVertices = station_df.distinct()
tripEdges = trip_df.withColumnRenamed("start_station_name", "src").withColumnRenamed("end_station_name", "dst")

In [7]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
stationVertices.limit(100).toPandas()

,id,name,lat,long,dock_count,city,installation_date
0,31,San Antonio Shopping Center,37.400443,-122.10833799999999,15,Mountain View,12/31/2013
1,25,Stanford in Redwood City,37.48537,-122.20328799999999,15,Redwood City,8/12/2013
2,83,Mezes Park,37.491269,-122.23623400000001,15,Redwood City,2/20/2014
3,21,Franklin at Maple,37.481758,-122.226904,15,Redwood City,8/12/2013
4,68,Yerba Buena Center of the Arts (3rd @ Howard),37.784878000000006,-122.40101399999999,19,San Francisco,8/23/2013
5,23,San Mateo County Center,37.487615999999996,-122.229951,15,Redwood City,8/15/2013
6,33,Rengstorff Avenue / California Street,37.400240999999994,-122.099076,15,Mountain View,8/16/2013
7,29,San Antonio Caltrain Station,37.406940000000006,-122.10675800000001,23,Mountain View,8/15/2013
8,46,Washington at Kearney,37.795425,-122.40476699999999,15,San Francisco,8/19/2013
9,49,Spear at Folsom,37.790302000000004,-122.39063700000001,19,San Francisco,8/20/2013


In [8]:
tripEdges.limit(1000).toPandas()

,id,duration,start_date,src,start_station_id,end_date,dst,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103
5,4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109
6,4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112
7,4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,95112
8,4760,113,8/29/2013 17:01,South Van Ness at Market,66,8/29/2013 17:03,South Van Ness at Market,66,553,Subscriber,94103
9,4258,114,8/29/2013 11:33,San Jose City Hall,10,8/29/2013 11:35,MLK Library,11,107,Subscriber,95060


In [7]:
stationGraph = GraphFrame(stationVertices, tripEdges)
tripEdges.cache()
stationVertices.cache()

DataFrame[id: string, name: string, lat: string, long: string, dock_count: string, city: string, installation_date: string]

In [8]:
print("Total Number of Stations: " + str(stationGraph.vertices.count()))
print("Total Number of Trips in Graph: " + str(stationGraph.edges.count()))
print("Total Number of Trips in Original Data: " + str(tripEdges.count())) # sanity check

Total Number of Stations: 70
Total Number of Trips in Graph: 669959
Total Number of Trips in Original Data: 669959


In [9]:
topTrips = stationGraph.edges.groupBy("src", "dst").count().orderBy("count", ascending=False).limit(10)
topTrips.limit(10).toPandas()

,src,dst,count
0,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th,6216
1,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,6164
2,Townsend at 7th,San Francisco Caltrain (Townsend at 4th),5041
3,2nd at Townsend,Harry Bridges Plaza (Ferry Building),4839
4,Harry Bridges Plaza (Ferry Building),2nd at Townsend,4357
5,Embarcadero at Sansome,Steuart at Market,4269
6,Embarcadero at Folsom,San Francisco Caltrain (Townsend at 4th),3967
7,Steuart at Market,2nd at Townsend,3903
8,2nd at South Park,Market at Sansome,3627
9,San Francisco Caltrain (Townsend at 4th),Harry Bridges Plaza (Ferry Building),3622


In [10]:
# get top 10 station in degrees
inDeg = stationGraph.inDegrees.orderBy("inDegree", ascending=False)
inDeg.limit(10).toPandas()

,id,inDegree
0,San Francisco Caltrain (Townsend at 4th),63179
1,San Francisco Caltrain 2 (330 Townsend),35117
2,Harry Bridges Plaza (Ferry Building),33193
3,Embarcadero at Sansome,30796
4,2nd at Townsend,28529
5,Market at Sansome,28033
6,Townsend at 7th,26637
7,Steuart at Market,25025
8,Temporary Transbay Terminal (Howard at Beale),23080
9,Market at 4th,19915


In [11]:
# get top 10 station out degrees
outDeg = stationGraph.outDegrees.orderBy("outDegree", ascending=False)
outDeg.limit(10).toPandas()

,id,outDegree
0,San Francisco Caltrain (Townsend at 4th),49092
1,San Francisco Caltrain 2 (330 Townsend),33742
2,Harry Bridges Plaza (Ferry Building),32934
3,Embarcadero at Sansome,27713
4,Temporary Transbay Terminal (Howard at Beale),26089
5,2nd at Townsend,25837
6,Steuart at Market,24838
7,Market at Sansome,24172
8,Townsend at 7th,23724
9,Market at 10th,20272


In [12]:
from pyspark.sql.functions import col

# get 10 stations with very high indegree to outdegree ratio (aka locations that are a sink with people going in and not out)
degreeRatio = inDeg.alias('a').join(outDeg.alias('b'), col('a.id') == col('b.id')).drop("b.id").selectExpr("a.id", "double(inDegree)/double(outDegree) as degreeRatio")
degreeRatio.cache()
degreeRatio.orderBy("degreeRatio", ascending=False).limit(10).toPandas()

,id,degreeRatio
0,Redwood City Medical Center,1.453376
1,Redwood City Public Library,1.300469
2,San Francisco Caltrain (Townsend at 4th),1.286951
3,Washington at Kearny,1.272367
4,MLK Library,1.233038
5,SJSU 4th at San Carlos,1.228205
6,San Mateo County Center,1.219512
7,Broadway at Main,1.208955
8,University and Emerson,1.205688
9,Washington at Kearney,1.203804


In [13]:
# Create simple graph for shortest path demo

# Vertex DataFrame
v = sql.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
  ("d", "David", 29),
  ("e", "Esther", 32),
  ("f", "Fanny", 36),
  ("g", "Gabby", 60)
], ["id", "name", "age"])
# Edge DataFrame
e = sql.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
  ("f", "c", "follow"),
  ("e", "f", "follow"),
  ("e", "d", "friend"),
  ("d", "a", "friend"),
  ("a", "e", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [14]:
# Get shortest path distances from each vertex to a landmark
results = g.shortestPaths(landmarks=["a", "d"])
results.select("id", "distances").limit(100).toPandas()

,id,distances
0,g,{}
1,b,{}
2,e,"{'d': 1, 'a': 2}"
3,a,"{'d': 2, 'a': 0}"
4,f,{}
5,d,"{'d': 0, 'a': 1}"
6,c,{}
